In [ ]:
!pip install tensorflow-gpu==1.8.0 Keras==2.1.6

In [ ]:
!sudo apt-get install graphviz -y


In [ ]:
!pip install pydot


In [ ]:
from keras.utils.vis_utils import model_to_dot
import pydot

import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from IPython.display import SVG


In [ ]:
!ls data

In [ ]:
train_folder = "data/train"
valid_folder = "data/valid"

In [ ]:

IMAGE_SIZE=224
BATCH_SIZE=64
NUM_CLASSES=120

In [ ]:
!ls data/train/ | head

In [ ]:

train_generator = ImageDataGenerator(preprocessing_function=keras.applications.resnet50.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = train_batches.num_classes

In [ ]:
base_model = keras.applications.mobilenet.MobileNet()

In [ ]:
base_model.summary()

In [ ]:
x = base_model.layers[-6].output
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.summary()

In [ ]:
model.layers[:-1]

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
SVG(model_to_dot(model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))


In [ ]:
#model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n//BATCH_SIZE, 
                    validation_data=valid_batches
                    ,  validation_steps=valid_batches.n // BATCH_SIZE, epochs=15, verbose=2,workers=4, 
                   use_multiprocessing=True
                   
                   )


In [ ]:
?model.fit_generator

In [ ]:
train_generator.class_indices.keys()

In [ ]:
model.save('dog_breed.h5')

In [ ]:
with open('labels.txt', 'w') as file_handler:
    for item in train_generator.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
tra

In [ ]:
!gsutil cp /home/ubuntu/custom/deeplizrd/dog_breed.h5 gs://np-training-public/

In [ ]:
!pwd

In [ ]:

!ls /home/ubuntu/.kaggle/competitions/dog-breed-identification/train_folder/

